In [1]:
from util.loadSensor import get_data, get_Mydata, get_leaveOneDatasetOut

In [2]:
from scipy.stats import kurtosis, skew
from scipy.signal import find_peaks
import numpy as np

def statistical_features(arr):
    arr = arr.numpy()
    vmin = np.amin(arr)
    vmax = np.amax(arr)
    mean = np.mean(arr)
    std = np.std(arr)
    return vmin, vmax, mean, std

def shape_features(arr):
    arr = arr.numpy()
    skewness = skew(arr)
    kurt = kurtosis(arr)
    return skewness, kurt

In [6]:
import os
import scipy.io as sio
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

data_path = f"/scratch/project_2014260/data/LOO/LOO1/test"
for root, dirs, files in os.walk(data_path):
    subs = sorted(files)
#train_dataset,valid_dataset,test_dataset = get_Mydata(subs[0],data_path,0, target_sample=256*2)

In [38]:
___test = sio.loadmat(f'/scratch/project_2014260/data/LOO/LOO1/test/{subs[0]}')

In [36]:
for iii in ___test['x_data']:
    iii[0]=[0.0 for f in range(len(iii[0]))]
    break

In [39]:
___test['x_data'][0][0]

array([17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
       17., 16., 16., 17., 17., 17., 17., 17., 17., 16., 16., 16., 16.,
       17., 17., 17., 17., 17., 17., 18., 18., 18., 18., 18., 18., 18.,
       18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18., 18.,
       17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
       17., 17., 17., 16., 16., 16., 16., 17., 17., 17., 17., 17., 17.,
       18., 18., 18., 18., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
       17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
       17., 17., 17., 17., 16., 16., 17., 17., 17., 17., 17., 17., 16.,
       16., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
       17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
       17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
       17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17.,
       17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17., 17

In [12]:
### LEAVE ONE User OUT
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, confusion_matrix, cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone

LOO_list = [f'LOO{i}' for i in range(1, 13)]

methods = {
    'rf': RandomForestClassifier(n_estimators=100, max_depth=15),
    'lda': LinearDiscriminantAnalysis(),
    'qda': QuadraticDiscriminantAnalysis(),
    'logreg': LogisticRegression(solver='liblinear')
}

for method_name, base_clf in methods.items():
    data_accuracy = []
    data_precision = []
    data_specificity = []
    data_auroc = []
    data_kappa = []
    print(f"\n===== Running {method_name.upper()} =====\n")
    for loo in LOO_list:
        data_path = f"/scratch/project_2014260/data/LOO/{loo}/test"
        print('-----', data_path)

        avg_acc = []
        avg_prec = []
        avg_spec = []
        avg_auc = []
        avg_kappa = []

        for root, dirs, files in os.walk(data_path):
            subs = sorted(files)
        cutindex = 2 if loo == 'LOO3' else 5
        for i in subs[:cutindex]:
            print('User:', i)
            train_dataset, valid_dataset, test_dataset = get_Mydata(i, data_path, 0, target_sample=256*2)
            clf = clone(base_clf)
            
            x_train = []
            y_train = train_dataset.y.numpy()
            for epoch in train_dataset.x:
                features = []
                for sig in epoch:
                    sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
                    sig_skew, sig_kurtosis = shape_features(sig)
                    peaks, properties = find_peaks(sig, width=5)
                    num_Peaks = len(peaks)
                    prominences = np.array(properties['prominences'])
                    widths = np.array(properties['widths'])
                    amplitude = np.sum(prominences)
                    duration = np.sum(widths)
                    features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
                x_train.append(features)
            x_train = np.array(x_train)
            
            x_test = []
            y_test = test_dataset.y.numpy()
            for epoch in test_dataset.x:
                features = []
                for sig in epoch:
                    sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
                    sig_skew, sig_kurtosis = shape_features(sig)
                    peaks, properties = find_peaks(sig, width=5)
                    num_Peaks = len(peaks)
                    prominences = np.array(properties['prominences'])
                    widths = np.array(properties['widths'])
                    amplitude = np.sum(prominences)
                    duration = np.sum(widths)
                    features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
                x_test.append(features)
            x_test = np.array(x_test)

            try:
                clf.fit(x_train, y_train)
                y_pred = clf.predict(x_test)
                try:
                    y_prob = clf.predict_proba(x_test)[:, 1]
                except Exception:
                    y_prob = np.zeros_like(y_pred)
                
                # ACCURACY
                accuracy = accuracy_score(y_test, y_pred)
                avg_acc.append(accuracy)

                # PRECISION
                precision = precision_score(y_test, y_pred, zero_division=0)
                avg_prec.append(precision)

                # SPECIFICITY: TN / (TN + FP)
                cm = confusion_matrix(y_test, y_pred)
                specificity = cm[0,0] / (cm[0,0] + cm[0,1] + 1e-7)
                avg_spec.append(specificity)

                # AUROC
                try:
                    auc_roc = roc_auc_score(y_test, y_prob)
                except Exception:
                    auc_roc = np.nan
                avg_auc.append(auc_roc)

                # KAPPA
                kappa = cohen_kappa_score(y_test, y_pred)
                avg_kappa.append(kappa)

                print(f"User: {i} | Acc: {accuracy:.4f} | Prec: {precision:.4f} | Spec: {specificity:.4f} | AUC: {auc_roc:.4f} | Kappa: {kappa:.4f}")
            except Exception as e:
                print(f"Failed {method_name} on user {i}: {e}")

        data_accuracy.append(np.mean(avg_acc))
        data_precision.append(np.mean(avg_prec))
        data_specificity.append(np.mean(avg_spec))
        data_auroc.append(np.mean(avg_auc))
        data_kappa.append(np.mean(avg_kappa))

        print(f'LOO {loo}: Acc={data_accuracy[-1]:.4f}, Prec={data_precision[-1]:.4f}, Spec={data_specificity[-1]:.4f}, AUC={data_auroc[-1]:.4f}, Kappa={data_kappa[-1]:.4f}')

    results = pd.DataFrame({
        'LOO': LOO_list,
        'Accuracy': data_accuracy,
        'Precision': data_precision,
        'Specificity': data_specificity,
        'AUROC': data_auroc,
        'Kappa': data_kappa
    })
    
    print(f'MEAN: Acc={np.mean(data_accuracy):.4f}, Prec={np.mean(data_precision):.4f}, Spec={np.mean(data_specificity):.4f}, AUC={np.mean(data_auroc):.4f}, Kappa={np.mean(data_kappa):.4f}')
    results.to_csv(f'louo_{method_name}_results.csv', index=False)
    print(f"Results saved to louo_{method_name}_results.csv")
    print(results)


===== Running RF =====

----- /scratch/project_2014260/data/LOO/LOO1/test
User: sub_S10_Data.mat
User: sub_S10_Data.mat | Acc: 0.8750 | Prec: 1.0000 | Spec: 1.0000 | AUC: 1.0000 | Kappa: 0.7500
User: sub_S11_Data.mat
User: sub_S11_Data.mat | Acc: 0.6591 | Prec: 0.8889 | Spec: 0.9545 | AUC: 0.8037 | Kappa: 0.3182
User: sub_S13_Data.mat
User: sub_S13_Data.mat | Acc: 0.6667 | Prec: 0.8333 | Spec: 0.9167 | AUC: 0.6285 | Kappa: 0.3333
User: sub_S14_Data.mat
User: sub_S14_Data.mat | Acc: 0.8571 | Prec: 0.7778 | Spec: 0.7143 | AUC: 0.9807 | Kappa: 0.7143
User: sub_S15_Data.mat
User: sub_S15_Data.mat | Acc: 0.8182 | Prec: 0.8889 | Spec: 0.9091 | AUC: 0.8926 | Kappa: 0.6364
LOO LOO1: Acc=0.7752, Prec=0.8778, Spec=0.8989, AUC=0.8611, Kappa=0.5504
----- /scratch/project_2014260/data/LOO/LOO2/test
User: sub_subject0_Data.mat
User: sub_subject0_Data.mat | Acc: 0.7500 | Prec: 0.8095 | Spec: 0.8462 | AUC: 0.8683 | Kappa: 0.5000
User: sub_subject10_Data.mat
User: sub_subject10_Data.mat | Acc: 0.4853 

/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


User: sub_S10_Data.mat | Acc: 0.6250 | Prec: 0.6667 | Spec: 0.7500 | AUC: 0.6806 | Kappa: 0.2500
User: sub_S11_Data.mat


/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


User: sub_S11_Data.mat | Acc: 0.6818 | Prec: 0.7500 | Spec: 0.8182 | AUC: 0.8306 | Kappa: 0.3636
User: sub_S13_Data.mat


...

User: sub_UN_115_Data.mat | Acc: 0.7500 | Prec: 0.6667 | Spec: 0.5000 | AUC: 0.8889 | Kappa: 0.5000
User: sub_UN_116_Data.mat


/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


User: sub_UN_116_Data.mat | Acc: 0.6667 | Prec: 0.6000 | Spec: 0.3333 | AUC: 0.7222 | Kappa: 0.3333
User: sub_UN_117_Data.mat


/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


User: sub_UN_117_Data.mat | Acc: 0.6111 | Prec: 0.5769 | Spec: 0.3889 | AUC: 0.6574 | Kappa: 0.2222
LOO LOO12: Acc=0.7000, Prec=0.6414, Spec=0.4333, AUC=0.7481, Kappa=0.4000
MEAN: Acc=0.6138, Prec=0.6243, Spec=0.4310, AUC=0.6483, Kappa=0.2048
Results saved to louo_qda_results.csv
      LOO  Accuracy  Precision  Specificity     AUROC     Kappa
0    LOO1  0.674026   0.764815     0.828788  0.766728  0.348052
1    LOO2  0.674465   0.667941     0.573306  0.715610  0.348931
2    LOO3  0.626887   0.632508     0.605849  0.692943  0.253774
3    LOO4  0.675952   0.793939     0.666667  0.658833  0.332424
4    LOO5  0.599286   0.595789     0.236667  0.556185  0.147879
5    LOO6  0.495195   0.543364     0.158333  0.450864 -0.097963
6    LOO7  0.549692   0.549692     0.000000  0.500000  0.000000
7    LOO8  0.766667   0.749351     0.588889  0.805864  0.533333
8    LOO9  0.672222   0.603129     0.844444  0.861728  0.344444
9   LOO10  0.447875   0.477273     0.047222  0.438702 -0.120199
10  LOO11  0.48

In [14]:
### VANILLA TRANSFORMER
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.keras import TqdmCallback
import glob
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from sklearn import preprocessing
from collections import Counter
from numpy import where
from imblearn.over_sampling import SMOTE
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold
import keras.backend as K
import scipy

2025-07-22 10:43:10.243363: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-22 10:43:10.701091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753170190.797972 2811220 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753170190.802963 2811220 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753170191.322467 2811220 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [15]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [17]:
from sklearn.metrics import precision_score, roc_auc_score, confusion_matrix, cohen_kappa_score
import pandas as pd
import numpy as np

data_accuracy = []
data_precision = []
data_specificity = []
data_auroc = []
data_kappa = []

LOO_list = [f'LOO{i}' for i in range(1, 13)]

for loo in LOO_list:
    data_path = f"/scratch/project_2014260/data/LOO/{loo}/test"
    for root, dirs, files in os.walk(data_path):
        subs = sorted(files)

    cutindex = 2 if loo == 'LOO3' else 5

    avg_acc = []
    avg_prec = []
    avg_spec = []
    avg_auc = []
    avg_kap = []

    for i in subs[:cutindex]:
        print('-----',i)
        train_dataset,valid_dataset,test_dataset = get_Mydata(i,data_path,0, target_sample=256*2)
        train_data = {}
        test_data = {}
        train_data["data"] = train_dataset.x.numpy()
        train_data["label"] = train_dataset.y.numpy()
        test_data["data"] = test_dataset.x.numpy()
        test_data["label"] = test_dataset.y.numpy()

        num_transformer_blocks = 2
        head_size = 256
        num_heads = 4
        dropout = 0.25
        ff_dim = 4
        mlp_units=[128]
        mlp_dropout=0.4
  
        X_train = train_data['data'].astype(np.float32)
        y_train = train_data["label"]
        X_test = test_data['data'].astype(np.float32)
        y_test = test_data["label"]

        input = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
        x = input

        for _ in range(num_transformer_blocks):
            transformer_layer = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

        pooling_layer = layers.GlobalAveragePooling1D(data_format="channels_first")(transformer_layer)

        for dim in mlp_units:
            dense_layer = layers.Dense(dim, activation="relu")(pooling_layer)
            drop_out = layers.Dropout(mlp_dropout)(dense_layer)

        output_layer = keras.layers.Dense(2, activation="softmax")(drop_out)  # For binary classification

        model = tf.keras.Model(inputs=input, outputs=output_layer)
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                      loss="sparse_categorical_crossentropy",
                      metrics=['accuracy'])
        
        history = model.fit(X_train, y_train, epochs=30, batch_size=128,
                             validation_data=(X_test, y_test), verbose=0)
        scores = model.evaluate(X_test, y_test, batch_size=128, verbose=1)

        avg_acc.append(scores[1])

        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1)

        # Precision
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        avg_prec.append(prec)

        # Specificity
        cm = confusion_matrix(y_test, y_pred)
        specificity = cm[0,0] / (cm[0,0] + cm[0,1] + 1e-7)
        avg_spec.append(specificity)

        # ROC-AUC (for binary, so use y_pred_prob[:, 1])
        auc = roc_auc_score(y_test, y_pred_prob[:, 1])
        avg_auc.append(auc)

        # Kappa
        kappa = cohen_kappa_score(y_test, y_pred)
        avg_kap.append(kappa)

    # Average metrics for this LOO fold
    data_accuracy.append(np.mean(avg_acc))
    data_precision.append(np.mean(avg_prec))
    data_specificity.append(np.mean(avg_spec))
    data_auroc.append(np.mean(avg_auc))
    data_kappa.append(np.mean(avg_kap))
    print(f'LOO {loo}: Acc={data_accuracy[-1]:.4f}, Prec={data_precision[-1]:.4f}, Spec={data_specificity[-1]:.4f}, AUC={data_auroc[-1]:.4f}, Kappa={data_kappa[-1]:.4f}')

# SAVE TO CSV
results_df = pd.DataFrame({
    'LOO': LOO_list,
    'Accuracy': data_accuracy,
    'Precision': data_precision,
    'Specificity': data_specificity,
    'AUROC': data_auroc,
    'Kappa': data_kappa
})

results_df.to_csv('louo_transformer_results.csv', index=False)
print("Results saved to louo_transformer_results.csv")
print(results_df)

----- sub_S10_Data.mat


2025-07-22 12:04:26.051989: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 0.0995
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step

2025-07-22 12:04:44.989161: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
----- sub_S11_Data.mat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9773 - loss: 0.0435
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
----- sub_UN_116_Data.mat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5000 - loss: 3.3891
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step

2025-07-22 12:20:34.911280: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
----- sub_UN_117_Data.mat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5000 - loss: 1.3812
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step

2025-07-22 12:20:50.225172: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
LOO LOO12: Acc=0.6444, Prec=0.6305, Spec=0.5111, AUC=0.7074, Kappa=0.2889
Results saved to louo_transformer_results.csv
      LOO  Accuracy  Precision  Specificity     AUROC     Kappa
0    LOO1  0.804437   0.818633     0.796537  0.895245  0.608874
1    LOO2  0.579465   0.558997     0.612777  0.667281  0.158931
2    LOO3  0.759528   0.772904     0.872830  0.737288  0.519057
3    LOO4  0.510675   0.375764     0.592778  0.550309 -0.007222
4    LOO5  0.667738   0.671583     0.487222  0.709375  0.301816
5    LOO6  0.752944   0.678626     0.575000  0.796136  0.449175
6    LOO7  0.469020   0.331862     0.272222  0.667238 -0.077231
7    LOO8  0.555556   0.522584     0.477778  0.527160  0.111111
8    LOO9  0.788889   0.690000     1.000000  0.887654  0.577778
9   LOO10  0.595250   0.679313     0.463500  0.619487  0.220080
10  LOO11  0.622222   0.494231     0.511111  0.651235  0.244444
11  LOO12  0.644444   0.630506     0.511111  0.707407  0.288889


In [18]:
np.mean(results_df['Accuracy'])

0.6458473342160383

In [14]:
### LEAVE ONE DATASET OUT
## RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, confusion_matrix, cohen_kappa_score
import numpy as np
import pandas as pd

LOOs = ['LOO1','LOO2','LOO3','LOO4','LOO5','LOO6','LOO7','LOO8','LOO9','LOO10','LOO11','LOO12']

avg_acc = []
avg_prec = []
avg_auc = []
avg_spec = []
avg_kappa = []

for LOO in LOOs:
    test_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/test"
    train_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/finetune"
    train_dataset, valid_dataset, test_dataset = get_leaveOneDatasetOut(test_path, train_path, 0, target_sample=256*2)
    clf = RandomForestClassifier(n_estimators=100, max_depth=15)
        
    x_train = []
    y_train = train_dataset.y.numpy()
    for epoch in train_dataset.x:
        features = []
        for sig in epoch:
            sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
            sig_skew, sig_kurtosis = shape_features(sig)
            peaks, properties = find_peaks(sig, width=5)
            num_Peaks = len(peaks)
            prominences = np.array(properties['prominences'])
            widths = np.array(properties['widths'])
            amplitude = np.sum(prominences)
            duration = np.sum(widths)
            features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
        x_train.append(features)
    x_train = np.array(x_train)
    
    x_test = []
    y_test = test_dataset.y.numpy()
    for epoch in test_dataset.x:
        features = []
        for sig in epoch:
            sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
            sig_skew, sig_kurtosis = shape_features(sig)
            peaks, properties = find_peaks(sig, width=5)
            num_Peaks = len(peaks)
            prominences = np.array(properties['prominences'])
            widths = np.array(properties['widths'])
            amplitude = np.sum(prominences)
            duration = np.sum(widths)
            features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
        x_test.append(features)
    x_test = np.array(x_test)
    
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    y_prob = clf.predict_proba(x_test)[:,1]
    
    # ------- METRICS --------
    acc = accuracy_score(y_test, y_pred)
    avg_acc.append(acc)
    
    prec = precision_score(y_test, y_pred, zero_division=0)
    avg_prec.append(prec)
    
    auc = roc_auc_score(y_test, y_prob)
    avg_auc.append(auc)
    
    cm = confusion_matrix(y_test, y_pred)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1] + 1e-7)
    avg_spec.append(specificity)

    kappa = cohen_kappa_score(y_test, y_pred)
    avg_kappa.append(kappa)

    print(f"{LOO} -> Acc: {acc:.4f}, Prec: {prec:.4f}, AUC: {auc:.4f}, Spec: {specificity:.4f}, Kappa: {kappa:.4f}")
    
print("\nMean Accuracy:", np.mean(avg_acc))
print("Mean Precision:", np.mean(avg_prec))
print("Mean AUROC:", np.mean(avg_auc))
print("Mean Specificity:", np.mean(avg_spec))
print("Mean Kappa:", np.mean(avg_kappa))

results = pd.DataFrame({
    'LOO': LOOs,
    'Accuracy': avg_acc,
    'Precision': avg_prec,
    'AUROC': avg_auc,
    'Specificity': avg_spec,
    'Kappa': avg_kappa
})
results.to_csv('loo_rf_results.csv', index=False)
print("Saved results to loo_rf_results.csv")

LOO1 -> Acc: 0.5577, Prec: 0.6364, AUC: 0.4973, Spec: 0.8462, Kappa: 0.1154
LOO2 -> Acc: 0.5593, Prec: 0.5741, AUC: 0.5914, Spec: 0.6593, Kappa: 0.1185
LOO3 -> Acc: 0.5477, Prec: 0.5410, AUC: 0.5491, Spec: 0.4664, Kappa: 0.0954
LOO4 -> Acc: 0.7245, Prec: 0.7222, AUC: 0.7205, Spec: 0.6809, Kappa: 0.4467
LOO5 -> Acc: 0.7629, Prec: 0.7959, AUC: 0.8338, Spec: 0.7778, Kappa: 0.5254
LOO6 -> Acc: 0.5619, Prec: 0.6207, AUC: 0.6389, Spec: 0.5111, Kappa: 0.1105
LOO7 -> Acc: 0.5816, Prec: 0.6140, AUC: 0.6244, Spec: 0.5000, Kappa: 0.1491
LOO8 -> Acc: 0.4611, Prec: 0.4667, AUC: 0.4777, Spec: 0.3778, Kappa: -0.0778
LOO9 -> Acc: 0.6389, Prec: 0.6214, AUC: 0.6186, Spec: 0.5667, Kappa: 0.2778
LOO10 -> Acc: 0.4425, Prec: 0.4487, AUC: 0.4668, Spec: 0.6019, Kappa: -0.1000
LOO11 -> Acc: 0.7833, Prec: 0.7297, AUC: 0.8219, Spec: 0.6667, Kappa: 0.5667
LOO12 -> Acc: 0.5889, Prec: 0.5656, AUC: 0.6310, Spec: 0.4111, Kappa: 0.1778

Mean Accuracy: 0.6008557211928747
Mean Precision: 0.6113653180380132
Mean AUROC: 0

In [16]:
### LEAVE ONE DATASET OUT
## LDA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, confusion_matrix, cohen_kappa_score
import numpy as np
import pandas as pd

LOOs = ['LOO1','LOO2','LOO3','LOO4','LOO5','LOO6','LOO7','LOO8','LOO9','LOO10','LOO11','LOO12']

avg_acc = []
avg_prec = []
avg_auc = []
avg_spec = []
avg_kappa = []

for LOO in LOOs:
    test_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/test"
    train_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/finetune"
    train_dataset, valid_dataset, test_dataset = get_leaveOneDatasetOut(test_path, train_path, 0, target_sample=256*2)
    clf = LinearDiscriminantAnalysis()
        
    x_train = []
    y_train = train_dataset.y.numpy()
    for epoch in train_dataset.x:
        features = []
        for sig in epoch:
            sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
            sig_skew, sig_kurtosis = shape_features(sig)
            peaks, properties = find_peaks(sig, width=5)
            num_Peaks = len(peaks)
            prominences = np.array(properties['prominences'])
            widths = np.array(properties['widths'])
            amplitude = np.sum(prominences)
            duration = np.sum(widths)
            features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
        x_train.append(features)
    x_train = np.array(x_train)
    
    x_test = []
    y_test = test_dataset.y.numpy()
    for epoch in test_dataset.x:
        features = []
        for sig in epoch:
            sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
            sig_skew, sig_kurtosis = shape_features(sig)
            peaks, properties = find_peaks(sig, width=5)
            num_Peaks = len(peaks)
            prominences = np.array(properties['prominences'])
            widths = np.array(properties['widths'])
            amplitude = np.sum(prominences)
            duration = np.sum(widths)
            features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
        x_test.append(features)
    x_test = np.array(x_test)
    
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    y_prob = clf.predict_proba(x_test)[:,1]
    
    # ------- METRICS --------
    acc = accuracy_score(y_test, y_pred)
    avg_acc.append(acc)
    
    prec = precision_score(y_test, y_pred, zero_division=0)
    avg_prec.append(prec)
    
    auc = roc_auc_score(y_test, y_prob)
    avg_auc.append(auc)
    
    cm = confusion_matrix(y_test, y_pred)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1] + 1e-7)
    avg_spec.append(specificity)

    kappa = cohen_kappa_score(y_test, y_pred)
    avg_kappa.append(kappa)

    print(f"{LOO} -> Acc: {acc:.4f}, Prec: {prec:.4f}, AUC: {auc:.4f}, Spec: {specificity:.4f}, Kappa: {kappa:.4f}")
    
print("\nMean Accuracy:", np.mean(avg_acc))
print("Mean Precision:", np.mean(avg_prec))
print("Mean AUROC:", np.mean(avg_auc))
print("Mean Specificity:", np.mean(avg_spec))
print("Mean Kappa:", np.mean(avg_kappa))

results = pd.DataFrame({
    'LOO': LOOs,
    'Accuracy': avg_acc,
    'Precision': avg_prec,
    'AUROC': avg_auc,
    'Specificity': avg_spec,
    'Kappa': avg_kappa
})
results.to_csv('loo_lda_results.csv', index=False)
print("Saved results to loo_lda_results.csv")

LOO1 -> Acc: 0.5513, Prec: 0.6000, AUC: 0.6349, Spec: 0.7949, Kappa: 0.1026
LOO2 -> Acc: 0.6481, Prec: 0.6408, AUC: 0.6995, Spec: 0.6222, Kappa: 0.2963
LOO3 -> Acc: 0.5742, Prec: 0.5493, AUC: 0.5541, Spec: 0.3216, Kappa: 0.1484
LOO4 -> Acc: 0.7245, Prec: 0.6765, AUC: 0.7580, Spec: 0.5319, Kappa: 0.4401
LOO5 -> Acc: 0.7320, Prec: 0.6912, AUC: 0.8167, Spec: 0.5333, Kappa: 0.4479
LOO6 -> Acc: 0.5714, Prec: 0.6056, AUC: 0.5322, Spec: 0.3778, Kappa: 0.0974
LOO7 -> Acc: 0.6837, Prec: 0.6769, AUC: 0.6561, Spec: 0.5227, Kappa: 0.3455
LOO8 -> Acc: 0.4111, Prec: 0.4375, AUC: 0.3644, Spec: 0.2000, Kappa: -0.1778
LOO9 -> Acc: 0.7333, Prec: 0.6909, AUC: 0.7662, Spec: 0.6222, Kappa: 0.4667
LOO10 -> Acc: 0.4336, Prec: 0.4419, AUC: 0.5226, Spec: 0.5556, Kappa: -0.1209
LOO11 -> Acc: 0.7500, Prec: 0.6829, AUC: 0.8767, Spec: 0.5667, Kappa: 0.5000
LOO12 -> Acc: 0.6667, Prec: 0.6562, AUC: 0.6619, Spec: 0.6333, Kappa: 0.3333

Mean Accuracy: 0.623327097978595
Mean Precision: 0.6124825974106328
Mean AUROC: 0.

In [17]:
### LEAVE ONE DATASET OUT
## QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, confusion_matrix, cohen_kappa_score
import numpy as np
import pandas as pd

LOOs = ['LOO1','LOO2','LOO3','LOO4','LOO5','LOO6','LOO7','LOO8','LOO9','LOO10','LOO11','LOO12']

avg_acc = []
avg_prec = []
avg_auc = []
avg_spec = []
avg_kappa = []

for LOO in LOOs:
    test_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/test"
    train_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/finetune"
    train_dataset, valid_dataset, test_dataset = get_leaveOneDatasetOut(test_path, train_path, 0, target_sample=256*2)
    clf = QuadraticDiscriminantAnalysis()
        
    x_train = []
    y_train = train_dataset.y.numpy()
    for epoch in train_dataset.x:
        features = []
        for sig in epoch:
            sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
            sig_skew, sig_kurtosis = shape_features(sig)
            peaks, properties = find_peaks(sig, width=5)
            num_Peaks = len(peaks)
            prominences = np.array(properties['prominences'])
            widths = np.array(properties['widths'])
            amplitude = np.sum(prominences)
            duration = np.sum(widths)
            features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
        x_train.append(features)
    x_train = np.array(x_train)
    
    x_test = []
    y_test = test_dataset.y.numpy()
    for epoch in test_dataset.x:
        features = []
        for sig in epoch:
            sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
            sig_skew, sig_kurtosis = shape_features(sig)
            peaks, properties = find_peaks(sig, width=5)
            num_Peaks = len(peaks)
            prominences = np.array(properties['prominences'])
            widths = np.array(properties['widths'])
            amplitude = np.sum(prominences)
            duration = np.sum(widths)
            features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
        x_test.append(features)
    x_test = np.array(x_test)
    
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    y_prob = clf.predict_proba(x_test)[:,1]
    
    # ------- METRICS --------
    acc = accuracy_score(y_test, y_pred)
    avg_acc.append(acc)
    
    prec = precision_score(y_test, y_pred, zero_division=0)
    avg_prec.append(prec)
    
    auc = roc_auc_score(y_test, y_prob)
    avg_auc.append(auc)
    
    cm = confusion_matrix(y_test, y_pred)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1] + 1e-7)
    avg_spec.append(specificity)

    kappa = cohen_kappa_score(y_test, y_pred)
    avg_kappa.append(kappa)

    print(f"{LOO} -> Acc: {acc:.4f}, Prec: {prec:.4f}, AUC: {auc:.4f}, Spec: {specificity:.4f}, Kappa: {kappa:.4f}")

print("\nMean Accuracy:", np.mean(avg_acc))
print("Mean Precision:", np.mean(avg_prec))
print("Mean AUROC:", np.mean(avg_auc))
print("Mean Specificity:", np.mean(avg_spec))
print("Mean Kappa:", np.mean(avg_kappa))

results = pd.DataFrame({
    'LOO': LOOs,
    'Accuracy': avg_acc,
    'Precision': avg_prec,
    'AUROC': avg_auc,
    'Specificity': avg_spec,
    'Kappa': avg_kappa
})
results.to_csv('loo_qda_results.csv', index=False)
print("Saved results to loo_qda_results.csv")

/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


LOO1 -> Acc: 0.6346, Prec: 0.6400, AUC: 0.7160, Spec: 0.6538, Kappa: 0.2692


/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


LOO2 -> Acc: 0.6556, Prec: 0.6180, AUC: 0.6570, Spec: 0.4963, Kappa: 0.3111


/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


...

/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


LOO10 -> Acc: 0.5487, Prec: 0.5741, AUC: 0.5381, Spec: 0.5741, Kappa: 0.0991


/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


LOO11 -> Acc: 0.6500, Prec: 0.6116, AUC: 0.6568, Spec: 0.4778, Kappa: 0.3000
LOO12 -> Acc: 0.6333, Prec: 0.6111, AUC: 0.5928, Spec: 0.5333, Kappa: 0.2667

Mean Accuracy: 0.5881393544009742
Mean Precision: 0.58688697596534
Mean AUROC: 0.5836785659840257
Mean Specificity: 0.4761459059963172
Mean Kappa: 0.17269170384553537
Saved results to loo_qda_results.csv


/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, confusion_matrix, cohen_kappa_score
import numpy as np
import pandas as pd

LOOs = ['LOO1','LOO2','LOO3','LOO4','LOO5','LOO6','LOO7','LOO8','LOO9','LOO10','LOO11','LOO12']

avg_acc = []
avg_prec = []
avg_auc = []
avg_spec = []
avg_kappa = []

for LOO in LOOs:
    test_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/test"
    train_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/finetune"
    train_dataset, valid_dataset, test_dataset = get_leaveOneDatasetOut(test_path, train_path, 0, target_sample=256*2)
    clf = LogisticRegression(solver='liblinear')  # Good for binary/small datasets
        
    x_train = []
    y_train = train_dataset.y.numpy()
    for epoch in train_dataset.x:
        features = []
        for sig in epoch:
            sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
            sig_skew, sig_kurtosis = shape_features(sig)
            peaks, properties = find_peaks(sig, width=5)
            num_Peaks = len(peaks)
            prominences = np.array(properties['prominences'])
            widths = np.array(properties['widths'])
            amplitude = np.sum(prominences)
            duration = np.sum(widths)
            features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
        x_train.append(features)
    x_train = np.array(x_train)
    
    x_test = []
    y_test = test_dataset.y.numpy()
    for epoch in test_dataset.x:
        features = []
        for sig in epoch:
            sig_min, sig_max, sig_mean, sig_std = statistical_features(sig)
            sig_skew, sig_kurtosis = shape_features(sig)
            peaks, properties = find_peaks(sig, width=5)
            num_Peaks = len(peaks)
            prominences = np.array(properties['prominences'])
            widths = np.array(properties['widths'])
            amplitude = np.sum(prominences)
            duration = np.sum(widths)
            features += [sig_min, sig_max, sig_mean, sig_std, sig_skew, sig_kurtosis, num_Peaks, amplitude, duration]
        x_test.append(features)
    x_test = np.array(x_test)
    
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    y_prob = clf.predict_proba(x_test)[:,1]
    
    # ------- METRICS --------
    acc = accuracy_score(y_test, y_pred)
    avg_acc.append(acc)
    
    prec = precision_score(y_test, y_pred, zero_division=0)
    avg_prec.append(prec)
    
    auc = roc_auc_score(y_test, y_prob)
    avg_auc.append(auc)
    
    cm = confusion_matrix(y_test, y_pred)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1] + 1e-7)
    avg_spec.append(specificity)

    kappa = cohen_kappa_score(y_test, y_pred)
    avg_kappa.append(kappa)

    print(f"{LOO} -> Acc: {acc:.4f}, Prec: {prec:.4f}, AUC: {auc:.4f}, Spec: {specificity:.4f}, Kappa: {kappa:.4f}")

print("\nMean Accuracy:", np.mean(avg_acc))
print("Mean Precision:", np.mean(avg_prec))
print("Mean AUROC:", np.mean(avg_auc))
print("Mean Specificity:", np.mean(avg_spec))
print("Mean Kappa:", np.mean(avg_kappa))

results = pd.DataFrame({
    'LOO': LOOs,
    'Accuracy': avg_acc,
    'Precision': avg_prec,
    'AUROC': avg_auc,
    'Specificity': avg_spec,
    'Kappa': avg_kappa
})
results.to_csv('loo_logreg_results.csv', index=False)
print("Saved results to loo_logreg_results.csv")

LOO1 -> Acc: 0.5962, Prec: 0.7143, AUC: 0.6340, Spec: 0.8718, Kappa: 0.1923
LOO2 -> Acc: 0.6630, Prec: 0.6642, AUC: 0.6827, Spec: 0.6667, Kappa: 0.3259
LOO3 -> Acc: 0.5866, Prec: 0.5596, AUC: 0.5562, Spec: 0.3604, Kappa: 0.1731
LOO4 -> Acc: 0.7755, Prec: 0.7164, AUC: 0.7968, Spec: 0.5957, Kappa: 0.5442
LOO5 -> Acc: 0.7526, Prec: 0.7059, AUC: 0.7863, Spec: 0.5556, Kappa: 0.4904
LOO6 -> Acc: 0.5619, Prec: 0.6000, AUC: 0.5293, Spec: 0.3778, Kappa: 0.0800
LOO7 -> Acc: 0.6429, Prec: 0.6557, AUC: 0.6507, Spec: 0.5227, Kappa: 0.2674
LOO8 -> Acc: 0.4056, Prec: 0.4298, AUC: 0.3205, Spec: 0.2333, Kappa: -0.1889
LOO9 -> Acc: 0.6889, Prec: 0.6700, AUC: 0.7316, Spec: 0.6333, Kappa: 0.3778
LOO10 -> Acc: 0.4027, Prec: 0.3867, AUC: 0.4966, Spec: 0.5741, Kappa: -0.1772
LOO11 -> Acc: 0.7722, Prec: 0.7059, AUC: 0.8728, Spec: 0.6111, Kappa: 0.5444
LOO12 -> Acc: 0.6833, Prec: 0.6737, AUC: 0.6795, Spec: 0.6556, Kappa: 0.3667

Mean Accuracy: 0.6275994619139298
Mean Precision: 0.6235082324096888
Mean AUROC: 0

In [ ]:
### VANILLA TRANSFORMER
from sklearn.metrics import precision_score, roc_auc_score, confusion_matrix, cohen_kappa_score
import numpy as np

num_transformer_blocks = 2
head_size = 256
num_heads = 4
dropout = 0.25
ff_dim = 4
mlp_units=[128]
mlp_dropout=0.4

LOOs = ['LOO1','LOO2','LOO3','LOO4','LOO5','LOO6','LOO7','LOO8','LOO9','LOO10','LOO11','LOO12']

avg_acc = []
avg_prec = []
avg_auc = []
avg_spec = []
avg_kappa = []

for LOO in LOOs:
    test_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/test"
    train_path = f"/scratch/project_2014260/data/LOO_finetune/{LOO}/finetune"
    train_dataset, valid_dataset, test_dataset = get_leaveOneDatasetOut(test_path, train_path, 0, target_sample=256*2)

    train_data = {}
    test_data = {}
    
    train_data["data"] = train_dataset.x.numpy()
    train_data["label"] = train_dataset.y.numpy()
    test_data["data"] = test_dataset.x.numpy()
    test_data["label"] = test_dataset.y.numpy()
  
    X_train = train_data['data'].astype(np.float32)
    y_train = train_data["label"]
    X_test = test_data['data'].astype(np.float32)
    y_test = test_data["label"]
    
    input = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = input

    for _ in range(num_transformer_blocks):
        transformer_layer = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    pooling_layer = layers.GlobalAveragePooling1D(data_format="channels_first")(transformer_layer)

    for dim in mlp_units:
        dense_layer = layers.Dense(dim, activation="relu")(pooling_layer)
        drop_out = layers.Dropout(mlp_dropout)(dense_layer)

    output_layer = keras.layers.Dense(2, activation="softmax")(drop_out)  # <-- 2 for binary classification

    model = tf.keras.Model(inputs=input, outputs=output_layer)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                  loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    
    history = model.fit(X_train, y_train, epochs=30, batch_size=128, 
                         validation_data=(X_test, y_test), verbose=0) 
    scores = model.evaluate(X_test, y_test, batch_size=128, verbose=1)

    acc = scores[1]
    avg_acc.append(acc)

    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Precision
    prec = precision_score(y_test, y_pred, zero_division=0)
    avg_prec.append(prec)

    # ROC AUC (for binary)
    # y_test should be 0/1, y_pred_prob[:,1] is probability for class 1
    auc = roc_auc_score(y_test, y_pred_prob[:, 1])
    avg_auc.append(auc)

    # Specificity (for binary)
    cm = confusion_matrix(y_test, y_pred)
    specificity = cm[0,0] / (cm[0,0] + cm[0,1] + 1e-7)
    avg_spec.append(specificity)

    # Kappa
    kappa = cohen_kappa_score(y_test, y_pred)
    avg_kappa.append(kappa)

    print(f"{LOO} -> Acc: {acc:.4f}, Prec: {prec:.4f}, AUC: {auc:.4f}, Spec: {specificity:.4f}, Kappa: {kappa:.4f}")

print("\nMean Accuracy:", np.mean(avg_acc))
print("Mean Precision:", np.mean(avg_prec))
print("Mean AUROC:", np.mean(avg_auc))
print("Mean Specificity:", np.mean(avg_spec))
print("Mean Kappa:", np.mean(avg_kappa))

results = pd.DataFrame({
    'LOO': LOOs,
    'Accuracy': avg_acc,
    'Precision': avg_prec,
    'AUROC': avg_auc,
    'Specificity': avg_spec,
    'Kappa': avg_kappa
})
results.to_csv('loo_transformer_results.csv', index=False)
print("Saved results to loo_transformer_results.csv")